# Mobilenet Model

In [0]:
# Importing neccessary packages
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import os
import json
from utils import add_regularization, get_data_generators, plot_model_results
from Models import TrainingCheckpoint, ModelTrain

In [0]:
NUM_CLASSES = 11
with open('classes_list.json', 'r') as fp:
    classes_list = json.load(fp)

In [0]:
# Data augmentation for regularization and adding some extra training samples

datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    preprocessing_function=tf.keras.applications.vgg19.preprocess_input,
    rescale=1.0/255.0,
    horizontal_flip=True, 
    zoom_range=0.2,
    shear_range=0.2
)

In [0]:
TARGET_DIM = 224
BATCH_SIZE = 128

In [0]:
train_generator, validation_generator = get_data_generators(datagen, TARGET_DIM, BATCH_SIZE)

Found 9866 images belonging to 11 classes.
Found 3430 images belonging to 11 classes.


In [0]:
base_model = tf.keras.applications.mobilenet.MobileNet(
    include_top=False, 
    weights='imagenet', 
    input_shape=(TARGET_DIM, TARGET_DIM, 3)
)

In [0]:
base_model.summary()

Model: "mobilenet_1.00_224"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
conv1_pad (ZeroPadding2D)    (None, 225, 225, 3)       0         
_________________________________________________________________
conv1 (Conv2D)               (None, 112, 112, 32)      864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 112, 112, 32)      128       
_________________________________________________________________
conv1_relu (ReLU)            (None, 112, 112, 32)      0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 32)      288       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, 112, 112, 32

In [0]:
print('Layers in Mobilenet: ' + str(len(base_model.layers)))

Layers in Mobilenet: 87


In [0]:
preds = base_model.output
preds = tf.keras.layers.GlobalAveragePooling2D()(preds)
preds = tf.keras.layers.Dense(1024, activation=tf.nn.relu)(preds)
preds = tf.keras.layers.BatchNormalization()(preds)
preds = tf.keras.layers.Dense(512, activation=tf.nn.relu)(preds)
preds = tf.keras.layers.BatchNormalization()(preds)
preds = tf.keras.layers.Dense(256, activation=tf.nn.relu)(preds)
preds = tf.keras.layers.BatchNormalization()(preds)
preds = tf.keras.layers.Dense(128, activation=tf.nn.relu)(preds)
preds = tf.keras.layers.Dense(11, activation=tf.nn.softmax)(preds)

In [0]:
model = tf.keras.models.Model(base_model.input, preds)

In [0]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
conv1_pad (ZeroPadding2D)    (None, 225, 225, 3)       0         
_________________________________________________________________
conv1 (Conv2D)               (None, 112, 112, 32)      864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 112, 112, 32)      128       
_________________________________________________________________
conv1_relu (ReLU)            (None, 112, 112, 32)      0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 32)      288       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, 112, 112, 32)      128   

In [0]:
mobilenet = ModelTrain(model, TARGET_DIM, BATCH_SIZE)
mobilenet.freeze_layers(9)

In [0]:
mobilenet.model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
conv1_pad (ZeroPadding2D)    (None, 225, 225, 3)       0         
_________________________________________________________________
conv1 (Conv2D)               (None, 112, 112, 32)      864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 112, 112, 32)      128       
_________________________________________________________________
conv1_relu (ReLU)            (None, 112, 112, 32)      0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 32)      288       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, 112, 112, 32)      128   

In [0]:
mobilenet.model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss='categorical_crossentropy',
    metrics=['acc']
)

In [0]:
# Lets define checkpoint for model saving
filepath="./models/mobilenet-{epoch:02d}-{acc:.2f}.hdf5"
checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath, monitor='acc', verbose=1, save_best_only=True, mode='max')

training_checkpoint = TrainingCheckpoint()

csv_logger = tf.keras.callbacks.CSVLogger(filename='./logs/mobilenet_training.csv', append=True)

In [0]:
#Let's train the model 20 epochs as it will take a lot of time

mobilenet.model.fit_generator(
    train_generator,
    steps_per_epoch=train_generator.samples // BATCH_SIZE,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // BATCH_SIZE,
    callbacks=[checkpoint, csv_logger, training_checkpoint],
    epochs=60
)

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/60
77/77 [==============================] - ETA: 0s - loss: 0.9021 - acc: 0.7027
Epoch 00001: acc improved from -inf to 0.70271, saving model to ./drive/My Drive/food_11_weights/mobilenet-01-0.70.hdf5
77/77 [==============================] - 178s 2s/step - loss: 0.9021 - acc: 0.7027 - val_loss: 0.8894 - val_acc: 0.7073
Epoch 2/60
77/77 [==============================] - ETA: 0s - loss: 0.4913 - acc: 0.8390
Epoch 00002: acc improved from 0.70271 to 0.83898, saving model to ./drive/My Drive/food_11_weights/mobilenet-02-0.84.hdf5
77/77 [==============================] - 181s 2s/step - loss: 0.4913 - acc: 0.8390 - val_loss: 0.7182 - val_acc: 0.7656
Epoch 3/60
77/77 [==============================] - ETA: 0s - loss: 0.3915 - acc: 0.8687
Epoch 00003: acc improved from 0.83898 to 0.86866, saving model to ./drive/My Drive/food_11_weights/mobilenet-03-0.87.hdf5
77/77 [==============================] - 184s 2s/st

### Results

Training accuracy: 0.9062<br>
Training loss: 0.2687 <br>

Validation accuracy: 0.7749<br>
Validation loss: 0.7599<br>

Avoidable bias error: 0.0938 => 9.38%<br>
Variance error: 0.1401 => 14.99%<br>

### Conclusion:
Here variance error is higher than avoidable bias, so we need to regularize the model.

### Regularizing model

Let's use following techniques to regularize the model step by step:
1. Data augmentation. <br>
2. Adding weight regularization.<br>
3. Adding dropout. <br>


In [0]:
model = tf.keras.models.load_model('./models/mobilenet-05-0.91.hdf5')

In [0]:
datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    preprocessing_function=tf.keras.applications.vgg19.preprocess_input,
    rescale=1.0/255.0,
    horizontal_flip=True, 
    zoom_range=0.2,
    shear_range=0.2,
    width_shift_range=0.2,
    height_shift_range=0.2,
    rotation_range=30
)

In [27]:
train_generator, validation_generator = get_data_generators(datagen, TARGET_DIM, BATCH_SIZE)

Found 9866 images belonging to 11 classes.
Found 3430 images belonging to 11 classes.


In [0]:
# Lets define checkpoint for model saving
filepath="./models/mobilenet-regularized-{epoch:02d}-{val_acc:.2f}.hdf5"
checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')

training_checkpoint = TrainingCheckpoint()

csv_logger = tf.keras.callbacks.CSVLogger(filename='./logs/mobilenet_regularized.csv', append=True)

early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_acc', patience=3, mode='max')

## Kernel regularization

In [0]:
## Adding weight regularization to dense_1 and dense_3 layers.
weight_regularization_layers = [model.layers[-2], model.layers[-6]]
model = add_regularization(model, weight_regularization_layers, regularizer=tf.keras.regularizers.l2(l=0.003))

In [0]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss='categorical_crossentropy',
    metrics=['acc']
)

In [32]:
model.fit_generator(
    train_generator,
    steps_per_epoch=train_generator.samples // BATCH_SIZE,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // BATCH_SIZE,
    callbacks=[checkpoint, csv_logger, training_checkpoint, early_stopping],
    epochs=20
)

Epoch 1/20
77/77 [==============================] - ETA: 0s - loss: 2.4647 - acc: 0.8336
Epoch 00001: val_acc improved from -inf to 0.76052, saving model to ./drive/My Drive/food_11_weights/mobilenet-regularized-01-0.76.hdf5
77/77 [==============================] - 191s 2s/step - loss: 2.4647 - acc: 0.8336 - val_loss: 2.1598 - val_acc: 0.7605
Epoch 2/20
77/77 [==============================] - ETA: 0s - loss: 1.5820 - acc: 0.8471
Epoch 00002: val_acc did not improve from 0.76052
77/77 [==============================] - 188s 2s/step - loss: 1.5820 - acc: 0.8471 - val_loss: 1.7875 - val_acc: 0.7166
Epoch 3/20
77/77 [==============================] - ETA: 0s - loss: 1.2588 - acc: 0.8481
Epoch 00003: val_acc improved from 0.76052 to 0.77764, saving model to ./drive/My Drive/food_11_weights/mobilenet-regularized-03-0.78.hdf5
77/77 [==============================] - 189s 2s/step - loss: 1.2588 - acc: 0.8481 - val_loss: 1.4003 - val_acc: 0.7776
Epoch 4/20
77/77 [==============================

### Results

Training accuracy: 0.8481<br>
Training loss: 1.2588 <br>

Validation accuracy: 0.7776<br>
Validation loss: 1.4003<br>

Avoidable bias error: 0.1519 => 15.19%<br>
Variance error: 0.0705 => 7.05%<br>

### Conclusion:

Here bias error is higher than variance error, but as inceptionV3 model trained previously show that the validation accuracy can increase upto 82%. So adding dropout layers to regularize model.

In [0]:
model = tf.keras.models.load_model('./models/vgg19-regularized-18-0.78.hdf5')

In [0]:
preds = model.layers[-8].output
preds = tf.keras.layers.Dropout(rate=0.4)(preds)
preds = model.layers[-7](preds)
preds = model.layers[-6](preds)
preds = model.layers[-5](preds)
preds = model.layers[-4](preds)
preds = tf.keras.layers.Dropout(rate=0.3)(preds)
preds = model.layers[-3](preds)
preds = model.layers[-2](preds)
preds = model.layers[-1](preds)

In [0]:
model = tf.keras.models.Model(model.input, preds)

In [0]:
mobilenet = ModelTrain(model, TARGET_DIM, BATCH_SIZE)

In [0]:
mobilenet.freeze_layers(11)

In [0]:
mobilenet.model.compile(
    optimizer=tf.keras.optimizers.Adam(),
    loss = 'categorical_crossentropy',
    metrics = ['acc']
)

In [40]:
%%time
mobilenet.model.fit_generator(
    train_generator,
    steps_per_epoch=train_generator.samples // BATCH_SIZE,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // BATCH_SIZE,
    callbacks=[checkpoint, csv_logger, training_checkpoint],
    epochs=10
)

Epoch 1/10
77/77 [==============================] - ETA: 0s - loss: 1.2423 - acc: 0.7961
Epoch 00001: val_acc did not improve from 0.77764
77/77 [==============================] - 190s 2s/step - loss: 1.2423 - acc: 0.7961 - val_loss: 1.3529 - val_acc: 0.7428
Epoch 2/10
77/77 [==============================] - ETA: 0s - loss: 1.1435 - acc: 0.7966
Epoch 00002: val_acc improved from 0.77764 to 0.78906, saving model to ./drive/My Drive/food_11_weights/mobilenet-regularized-02-0.79.hdf5
77/77 [==============================] - 190s 2s/step - loss: 1.1435 - acc: 0.7966 - val_loss: 1.1545 - val_acc: 0.7891
Epoch 3/10
77/77 [==============================] - ETA: 0s - loss: 1.0671 - acc: 0.8012
Epoch 00003: val_acc did not improve from 0.78906
77/77 [==============================] - 193s 3s/step - loss: 1.0671 - acc: 0.8012 - val_loss: 1.1554 - val_acc: 0.7665
Epoch 4/10
77/77 [==============================] - ETA: 0s - loss: 1.0053 - acc: 0.8023
Epoch 00004: val_acc did not improve from 0.7

### Results

Training accuracy: 0.7966<br>
Training loss: 1.1435 <br>

Validation accuracy: 0.78906<br>
Validation loss: 1.1545<br>

Avoidable bias error: 0.2034 => 20.34%<br>
Variance error: 0.00754 => 0.754%<br>

### Conclusion:
Bias error can be reduced further but it won't help much as the validation accuracy is not increasing so model tends to overfit on training data.